# Codebase Overview

This notebook is intended to provide developers with orientation on how the different parts of the DeepTrack2 codebase work together, with a focus on the lower level modules and core functionalities. This tutorial should ideally be read in tandem with the [deeptrack/](https://github.com/DeepTrackAI/DeepTrack2/tree/develop/deeptrack) directory structure in order to provide perspective.

The following topics will be covered:

- Module structure

- Low-level modules such as `core` and `image`.

- `Features` module.

- `Image` module.

- High-level modules such as `holography` and `optics`.

- Utility and handle modules.


## 1. What is a module?

A module in the DeepTrack2 framework is a `.py` source code file 
containing classes and methods which implement functionalities in a _modular_ way to make maintenance and debugging easier. Inheritance is used heavily in the design pattern of DeepTrack2, and understanding the hierarchy is crucial


## 2. Module structure

The first thing in a module is always the documentation for the class which
goes over things like key features, structure, examples.

The first class in a module is typically an **abstract** class. 

This abstract class provides a standardized implementation of core functionality (e.g., data processing) to ensure consistent output formatting across all derived classes in the module. For instance, whether `class A` or `class B` produces the output, the resulting format remains the same.


A typical module is structured like the following:

```python

"""Example module.

"""

from .backend import BackendClass


# Define abstract class that inherits from some backbone module.
class AbstractClass(BackendClass):
    """Serves as an abstract class.
    """

    def __init__(...):
        super().__init__(...)
    
    def process():
        self.get()

class AdditionCase(AbstractClass):
    """Defines addition.
    """
    
    def __init__(...)
        super().__init__(...)
    
    # Define abstract method.
    def get(...)
        return 1+1

```

## 3. Type Hints for Code Readability with [types](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/types.py) module

As type hints are used extensively to improve code readability in accordance with the [style guide](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/tutorials/4-developers/DTDV411_style.ipynb), DeepTrack2 introduces a few custom type hints for internal use, declared in the [types](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/types.py) module.

Currently there are three type aliases:

- `PropertyLike`: Alias representing either a value of generic type `T` or a callable function returning a value of generic type `T`.

- `ArrayLike`: Alias for array-like structures (e.g., tuples, lists, numpy arrays).

- `NumberLike`: Alias for numeric types, including scalars and arrays (e.g., numpy 
    arrays, GPU tensors).

And one would incorporate these type hints like the following:
```python
from .types import ArrayLike, PropertyLike

def get(
        self,
        image: ArrayLike,
        uses: PropertyLike[int],
        storage: PropertyLike[int],
        **kwargs
) -> List[ArrayLike]:
```



## 4. Low level modules.

This section covers the low-level modules in DeepTrack2 found in [backend/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/) and [sources/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/). The contents of the modules are classes and methods which have simple but important
functionalities that form the base for higher level modules.


### 4.1. The backbone of DeepTrack2

In the [backend/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/) directory is the code that forms the backbone of the framework, which consists of five modules:

- [core](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/core.py): Provides the core DeepTrack2 classes to manage and process data on a fundamental level. 

More specifically, [core](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/core.py) introduces the `DeepTrackNode` class which is used to represent a node in a computation graph, which when used together with the `DeepTrackDataObject` class can store data and compute new data based on its dependencies and child nodes. Inherent to these classes is utility for dependency tracking and data validation with ID and index addresses.

- [mie](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/mie.py): Provides functions to perform Mie scattering calculations used in simulations.

In particular computing coefficients for both spherical and stratified spherical harmonics, and calculating the spherical harmonics of the Mie field with an iterative method.

- [pint_definition](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/pint_definition.py): Extends Pint's default definitions by introducing project-specific constants
    and unit modification for flexible calculations in he context of DeepTrack2.

- [polynomials](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/polynomials.py): Provides a set of functions which compute Bessel and Riccati-Bessel polynomials and their derivatives.

- [units](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/units.py): Provides unit management and conversions for simulations involving voxel dimensions and scaling factors.

The [core](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/backend/core.py) also forms the base class for the [features](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/features.py) module which is the biggest module in DeepTrack2 in terms of code volume, and in turn forms the base class for all other modules in the deeptrack directory, the only exceptions being [image](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/image.py) and [properties](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/properties.py). 

### 4.2. Managing data sources

In the [sources/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/) directory is the code used to provide utility classes to manage image data sources and random number generators, the modules are:

- [base](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/base.py): Extends DeepTrackNode objects to represent sources of data, and enables data validity checking.

- [folder](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/folder.py): Extends the [base](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/base.py) module by introducing utilities to organize sources in directories with labeling and source splitting.

- [rng](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/sources/rng.py): Extends both the standard library rng and NumPy rng to let the user instance as many generators as desired with unique seeds.


## 5.  Data Structures and Transformations with [features](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/features.py) 

This module introduces the `Feature` and `StructuralFeature` base classes, which form the base for all 
features and implementations in DeepTrack2. A `Feature` is a building block of a data processing pipeline, and it represents a transformation applied to data. For example in the the [augmentations](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/augmentations.py) module: an augmentation that rotates an image would be implemented as a subclass to `Feature`.

The `StructuralFeature` class further extends `Feature` by adding
logical structures, such as branches or chaining. It enables the construction of data pipelines with more advanced requirements.

### 5.1. What is a feature?
A feature in the literal sense is defined as a transformation of an image. Some examples of these transformations include but is not limited to:

- Rotations or deformations
- Noise addition or background illumination
- Non-additive elements, such as Poisson noise


## 6. Adding Attributes and Properties to Features with [properties](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/properties.py) 

This module introduces the `Property`, `SequentialProperty` and `PropertyDict` base classes, which when used in combination with the [features](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/features.py) module lets the user add values to a `Feature`. These values have no datatype restrictions and can represent constants, functions, lists, `Image` etc. `PropertyDict` in turn represents a dictionary with `Property` elements.

`SequentialProperty` extends the `Property` class to enables sequential updates to handle scenarios where the property’s value evolves over discrete steps, such as frames in a video or time-series data.



## 7. Containers for Array-Like Structures with [image](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/image.py)

This module only contains a single class, namely `Image`. This class serves as a wrapper for array-like data and provides a unified interface for array operations and property management with NumPy and CuPy compatibility.

Several utility functions are also included, which can be used to manipulate `Image` objects within pipelines, such as image coercion to ensure a consistent type across a series of images, padding to optimize Fast Fourier Transform performance, or conversion to CuPy arrays with GPU-based acceleration.


## 8. High level modules

The following modules jointly implement the main functionality of DeepTrack2, which is synthetic data generation using simulations. All the classes in the following modules extend `Feature` and utilize `Image` objects as containers, as all of these will represent transformations in some way:

- [aberrations](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/aberrations.py): Framework to simulate optical aberrations in microscopy setups by implementing Zernike polynomials and Gaussian pupil apodization.

- [augmentations](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/augmentations.py): Framework to augment data with various transformations and image manipulation techniques such as rotations, deformations, cropping, padding.

- [holography](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/holography.py): Provides features for managing optical fields with Fourier transforms and propagation matrices for usage in simulation pipelines and holographic reconstructions.

- [noises](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/noises.py): Framework to add various types of additive noise to images as well as Poisson noise based on signal-to-noise ratio.

- [optics](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/optics.py): Framework for simulating the optics of a variety of microscopy setups such as darkfield, fluorescence, or brightfield, as well as incorporating the interaction between a sample (often a scatterer object) and a microscopy setup.

- [scatterers](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/scatterers.py): Framework for implementing light-scattering objects, 





## 9. Handle and utility modules

The following modules play a mediating role between external libraries and DeepTrack2, such as integrating Pytorch classes or Numpy functions, as well as utilities for type consistency and radial center calculations.

### 9.1. Numpy
The following modules provides handles to enables the integration of NumPy functions to be used on DeepTrack2 objects as well as mathematical utilities:

- [elementwise](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/elementwise.py): Provides handles to perform various elementary NumPy functions elementwise to `Feature` objects, as well as sequentially. Some of these functions include trigonometric, hyperbolic, rounding, exponents and more.

- [math](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/math.py): Provides utilities for various types of normalization, blurring, pooling, and resizing of `Image` objects.

- [statistics](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/statistics.py): Provides handles to perform various statistical NumPy functions on a given `Feature` objects along a given axis. 

### 9.2. Pytorch
Located in the [pytorch/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/pytorch/) directory are two modules to facilitate PyTorch integration with DeepTrack2 objects:

- [pytorch/data](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/pytorch/data.py): Extends the PyTorch [Dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class to work with DeepTrack2 `Image` objects.

- [pytorch/features](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/pytorch/features.py): Extends `Feature` to be able to convert an input to a PyTorch [Tensor](https://pytorch.org/docs/stable/tensors.html).

### 9.3. Deeplay
The init file in [deeplay/__init__.py](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/deeplay/__init__.py) enables users to import Deeplay from DeepTrack2.\
with:
```python 
import deeptrack.deeplay as dl
```

### 9.4. Misc. modules

- [utils](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/utils.py): Provides various utilities to streamline common operations, ensuring type and argument consistency with various check methods and safe call

- [extras/radialcenter](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/extras/radialcenter.py): Introduces a single function to calculate the center location of an intensity distribution with a least-squares method.



## 10. Unit testing

The unit testing scritps are found in the [tests/](https://github.com/DeepTrackAI/DeepTrack2/blob/develop/deeptrack/tests/) directory, and follow the same structure as the [deeptrack/](https://github.com/DeepTrackAI/DeepTrack2/tree/develop/deeptrack) directory with a test script for each module, as well as subdirectories for these modules should they have one (e.g. [tests/extras/test_radialcenter](https://github.com/DeepTrackAI/DeepTrack2/tree/develop/deeptrack/tests/extras/test_radialcenter.py)).


